# Modelos de regressão com árvores

In [ ]:
!pip install catboost

In [ ]:
! apt-get install default-jre
!java -version

In [ ]:
! pip install h2o

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import time

#training aux
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

#models
from sklearn import tree
import h2o
from catboost import CatBoostRegressor
from h2o.estimators import H2ORandomForestEstimator

In [14]:
#df = pd.read_csv('data/ENEM_CLEAN.csv')
path = '/content/drive/MyDrive/analise_enem/data/ENEM_CLEAN_WITH_NAN.csv'
df = pd.read_csv(path)
df = df.loc[:, ~(df == 'FALTANTE').any()]

## Modelos com árvore de decisão

In [16]:
#models with numeric variables
X = df[[col for col in df.columns if col[0:3] == 'NUM']].drop(columns = ['NUM_NOTA'])
X_columns_names = X.columns
X = X.values
Y = df.NUM_NOTA.values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

results_path = '/content/drive/MyDrive/analise_enem/results/decision_tree_numeric.csv'
with open(results_path, "w+") as f:
    f.write("max_depth;min_samples_split;min_samples_leaf;time;data;score")

i = 0
for max_depth in [10, 15, 20]:
    for min_samples_split in [20, 30, 40]:
        for min_samples_leaf in [30, 45, 60]:
            start = time.time()
            model = tree.DecisionTreeRegressor(max_depth= max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf)
            
            cv_scores = cross_val_score(model, x_train, y_train, cv = 3)
            end = time.time()
            with open(results_path, "a") as f:
                f.write(f"\n{max_depth};{min_samples_split};{min_samples_leaf};{(end - start)/3:.4f};train;{cv_scores.mean()}")
            i+= 1
            if i % 25 == 0:
                print(f"On iteration {i}.")

decision_tree_numeric_results = pd.read_csv(results_path, sep = ";").sort_values('score', ascending = False)
max_depth = decision_tree_numeric_results.max_depth.iloc[0]
min_samples_split = decision_tree_numeric_results.min_samples_split.iloc[0]
min_samples_leaf = decision_tree_numeric_results.min_samples_leaf.iloc[0]

start = time.time()
model = tree.DecisionTreeRegressor(max_depth= max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf)

model.fit(x_train, y_train)
end = time.time()

with open(results_path, "a") as f:
    f.write(f"\n{max_depth};{min_samples_split};{min_samples_leaf};{end - start:.4f};test;{model.score(x_test, y_test)}")

On iteration 25.


In [19]:
#models with numeric variables and categorics
X = pd.get_dummies(df, drop_first = True, columns = [col for col in df.columns if col[0:3] == 'CAT']).drop(columns = ['NUM_NOTA'])
X_columns_names = X.columns
X = X.values
Y = df.NUM_NOTA.values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

results_path = '/content/drive/MyDrive/analise_enem/results/decision_tree_numeric_categoric.csv'
with open(results_path, "w+") as f:
    f.write("max_depth;min_samples_split;min_samples_leaf;time;data;score")

i = 0
for max_depth in [10, 15, 20]:
    for min_samples_split in [20, 30, 40]:
        for min_samples_leaf in [30, 45, 60]:
            start = time.time()
            model = tree.DecisionTreeRegressor(max_depth= max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf,
                                              )
            
            cv_scores = cross_val_score(model, x_train, y_train, cv = 3)
            end = time.time()
            with open(results_path, "a") as f:
                f.write(f"\n{max_depth};{min_samples_split};{min_samples_leaf};{(end - start)/3:.4f};train;{cv_scores.mean()}")
            i+= 1
            if i % 25 == 0:
                print(f"On iteration {i}.")

decision_tree_numeric_categoric_results = pd.read_csv(results_path, sep = ";").sort_values('score', ascending = False)
max_depth = decision_tree_numeric_categoric_results.max_depth.iloc[0]
min_samples_split = decision_tree_numeric_categoric_results.min_samples_split.iloc[0]
min_samples_leaf = decision_tree_numeric_categoric_results.min_samples_leaf.iloc[0]

start = time.time()
model = tree.DecisionTreeRegressor(max_depth= max_depth,
                                    min_samples_split = min_samples_split,
                                    min_samples_leaf = min_samples_leaf)

model.fit(x_train, y_train)
end = time.time()

with open(results_path, "a") as f:
    f.write(f"\n{max_depth};{min_samples_split};{min_samples_leaf};{end - start:.4f};test;{model.score(x_test, y_test)}")

On iteration 25.


## Modelo CatBoost

In [22]:
#models with numeric variables and categorics
X = df.drop(columns = ['NUM_NOTA'])
X_columns_names = X.columns
#X = X.values
Y = df.NUM_NOTA.values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

results_path = '/content/drive/MyDrive/analise_enem/results/catboost_numeric_categoric.csv'
with open(results_path, "w+") as f:
    f.write("iterations;learning_rate;depth;time;data;score")

i = 0
for iterations in [400, 500, 650]:
    for learning_rate in [0.1, 0.2, 0.5]:
        for depth in [6, 10, 14]:
            start = time.time()
            model = CatBoostRegressor(iterations = iterations,
                                      learning_rate = learning_rate,
                                      depth = depth,
                                      verbose = False,
                                      cat_features = [col for col in X_columns_names if col[0:3] == 'CAT'],
                                      random_state = 1,
                                      task_type='GPU'
                                      )            
            end = time.time()
            cv_scores = cross_val_score(model, x_train, y_train, cv = 3)

            with open(results_path, "a") as f:
                f.write(f"\n{iterations};{learning_rate};{depth};{(end - start)/3:.4f};train;{cv_scores.mean()}")
            i+= 1
            if i % 5 == 0:
                print(f"On iteration {i}.")

catboost_results = pd.read_csv(results_path, sep = ";").sort_values('score', ascending = False)
iterations = catboost_results.iterations.iloc[0]
learning_rate = catboost_results.learning_rate.iloc[0]
depth = catboost_results.depth.iloc[0]

start = time.time()
model = CatBoostRegressor(iterations = iterations,
                               learning_rate = learning_rate,
                               depth = depth,
                               verbose = False,
                               cat_features = [col for col in X_columns_names if col[0:3] == 'CAT'],
                               random_state = 1,
                               task_type='GPU'
                              )            

model.fit(x_train, y_train)
end = time.time()

with open(results_path, "a") as f:
    f.write(f"\n{iterations};{learning_rate};{depth};{end - start:.4f};test;{model.score(x_test, y_test)}")

On iteration 5.
On iteration 10.
On iteration 15.
On iteration 20.
On iteration 25.


In [23]:
catboost_results

,iterations,learning_rate,depth,time,data,score
25,650,0.5,10,0.0007,train,0.308283
23,650,0.2,14,0.0001,train,0.305837
26,650,0.5,14,0.0001,train,0.305243
24,650,0.5,6,0.0001,train,0.304569
16,500,0.5,10,0.0001,train,0.303416
22,650,0.2,10,0.0001,train,0.299155
7,400,0.5,10,0.0001,train,0.296886
13,500,0.2,10,0.0003,train,0.296322
17,500,0.5,14,0.0001,train,0.296167
6,400,0.5,6,0.0001,train,0.293811


## Modelo Distributed Random Forest

In [41]:
h2o.init()

h2o_df = h2o.H2OFrame(df)
for col in h2o_df.columns:
    if col[0:3] == 'CAT':
        h2o_df[col] = h2o_df[col].asfactor()

df_train, df_test = h2o_df.split_frame(ratios=[.8], seed=1)


results_path = '/content/drive/MyDrive/analise_enem/results/drf_numeric_categoric.csv'
with open(results_path, "w+") as f:
    f.write("ntrees;max_depth;time;score")


i = 0
for ntrees in [25, 50, 75]:
    for max_depth in [15, 20, 25]:
        start = time.time()
        model = H2ORandomForestEstimator(
                                        nfolds = 3,
                                        ntrees = ntrees,
                                        max_depth = max_depth,
                                        seed = 1,
                                        )            
        model.train(x = [col for col in h2o_df.columns if col != 'NUM_NOTA'],
                    y = 'NUM_NOTA',
                    training_frame = df_train,
                    validation_frame = df_test)
        end = time.time()

        with open(results_path, "a") as f:
            f.write(f"\n{ntrees};{max_depth};{end-start:.4f};{model.r2(valid = True)}")
        i+= 1
        if i % 5 == 0:
            print(f"On iteration {i}.")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,30 mins 57 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_unknownUser_w3qhi0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.165 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [CAT_CADEIRA_ACOLCHOADA, CAT_LACTANTE, CAT_GUIA_INTERPRETE, CAT_IDOSO, CAT_BRAILLE, CAT_MATERIAL_ESPECIFICO, CAT_SOROBAN, CAT_CEGUEIRA, CAT_LEITURA_LABIAL, CAT_CADEIRA_CANHOTO, CAT_SURDEZ, CAT_VISAO_MONOCULAR, CAT_MOBILIARIO_OBESO, CAT_MAQUINA_BRAILE, CAT_SONDA, CAT_LIBRAS, CAT_COMPUTADOR, CAT_AMPLIADA_18, CAT_NOME_SOCIAL, CAT_DEFICIENCIA_AUDITIVA, CAT_AUTISMO, CAT_MESA_CADEIRA_SEPARADA, CAT_ESTUDA_CLASSE_HOSPITALAR, CAT_SALA_ACOMPANHANTE, CAT_MOBILIARIO_ESPECIFICO, CAT_PROTETOR_AURICULAR, CAT_SALA_ESPECIAL, CAT_MEDIDOR_GLICOSE, CAT_PROVA_DEITADO, CAT_LAMINA_OVERLAY, CAT_GESTANTE, CAT_SALA_INDIVIDUAL, CAT_SURDO_CEGUEIRA, CAT_MARCA_PASSO, CAT_MEDICAMENTOS, CAT_APOIO_PERNA, CAT_CADEIRA_ESPECIAL]
  warnings.warn(mesg["message"], RuntimeWarning)


drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
On iteration 5.
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
